In [1]:
%%time
import glob
import numpy as np
import pandas as pd
# 显示3位小数
pd.set_option('display.float_format',lambda x: '%.3f'% x )
path ="E:/sjzc/hour"
df0=pd.read_excel("E:/sjzc/质差小区列表_洛阳_unsolved.xlsx")
filenames = glob.glob(path + "/*.csv")
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename,encoding="gbk"))
# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs, ignore_index=True)
a=pd.merge(df0,big_frame,how="left",on="ECI")
# 获取所有http和video数据
r = a.iloc[:,[0,1,2,3,4,5,6,7,8,9,12,28,29,30,31,32,33,42,43,44]]
r.to_excel("E:/sjzc/hour/LTE业务感知指标_详表.xlsx",index=False)
http=r["HTTP大包下载总业务量(MB)"].groupby(r["ECI"]).sum()/r["HTTP大包下载总时长(s)"].groupby(r["ECI"]).sum()*8
video=r["视频业务下载总业务量(MB)"].groupby(r["ECI"]).sum()/r["视频业务下载总时长(S)"].groupby(r["ECI"]).sum()*8
http_hd=r["HTTP大包业务低速率话单数(小于2.5Mbps话单数)"].groupby(r["ECI"]).sum()/r["HTTP大包业务总话单数"].groupby(r["ECI"]).sum()*100
# 将series变成DateFrame
http_pd=pd.DataFrame(http)
video_pd=pd.DataFrame(video)
http_hd_pd=pd.DataFrame(http_hd)
# 对DateFrame列重命名
http_pd.rename(columns={0:"http下载速率(Mbps)"},inplace =  True)
video_pd.rename(columns={0:"视频业务下载速率(Mbps)"},inplace =  True)
http_hd_pd.rename(columns={0:"HTTP大包业务低速率话单占比(%)"},inplace =  True)
# 生成总的小区速率v
cell_v=http_pd.merge(video_pd,how="left",on="ECI")
cell_v_hd=cell_v.merge(http_hd_pd,how="left",on="ECI")
cell_v_hd_hb=pd.merge(cell_v_hd,df0,how="left",on="ECI")
# s = cell_v_hb.iloc[:,[0,3,4,5,6,7,8,9,10,11,1,2,]]
s =  cell_v_hd_hb.iloc[:,[0,4,5,8,10,11,12,1,2,3]]
# 定义质差标准
nx=1.5
xs=2
nx_hd=20
xs_hd=10
# 自定义函数，获取是否质差的情况
def function(a,b,c,d,e):
    if (a == '视频质差')and (b == '农村'or b == '乡镇') and d < nx :
        return 1
    elif (a == '视频质差')and (b == '县城'or b == '市区') and d < xs :
        return 1
    elif (a == 'http质差')and (b == '农村'or b == '乡镇') and c < nx :
        return 1
    elif (a == 'http质差')and (b == '县城'or b == '市区') and c < xs :
        return 1
    elif (a == '视频质差&http质差')and (b == '农村'or b == '乡镇') and (c<nx or d <nx):
        return 1
    elif (a == '视频质差&http质差')and (b == '县城'or b == '市区') and (c<xs or d <xs):
        return 1
    elif (a == 'http质差&视频质差')and (b == '农村'or b == '乡镇') and (c<nx or d <nx):
        return 1
    elif (a == 'http质差&视频质差')and (b == '县城'or b == '市区') and (c<xs or d <xs):
        return 1
    elif (a == 'http质差话单占比高')and (b == '农村'or b == '乡镇') and (e>nx_hd):
        return 1      
    elif (a == 'http质差话单占比高')and (b == '县城'or b == '市区') and (e>xs_hd):
        return 1 
    else:
        return 0
s['是否质差']=s.apply(lambda x : function(x['质差类型'],x['归属区域'],x['http下载速率(Mbps)'],x['视频业务下载速率(Mbps)'],x['HTTP大包业务低速率话单占比(%)']),axis=1)
s.set_index("ECI",inplace = True)
s.to_excel("E:/sjzc/hz_h.xlsx")
# 输出表头
size=s.shape[0]
print(a.sort_values(by='开始时间',ascending=False)['开始时间'].dropna().unique()[0],f"监控小区数{size}个")
# 对结果进行降序处理
display(s.sort_values(by=['是否质差','视频业务下载速率(Mbps)'],ascending=False,na_position='first'))
# 筛选出最新时段小时级指标
zc_h=r[r['开始时间']==a.sort_values(by='开始时间',ascending=False)['开始时间'].dropna().unique()[0]]
zc_h.set_index('ECI',inplace=True)
zc_h['是否质差']=zc_h.apply(lambda x : function(x['质差类型'],x['归属区域_x'],x['HTTP大包下载速率(Mbps)'],x['视频业务下载速率(Mbps)'],x['HTTP大包业务低速率话单占比(%)']),axis=1)
zc_h=zc_h.loc[:,['ENB_CI','CELLNAME','质差类型','归属区域_x','开始时间','HTTP大包下载总业务量(MB)','HTTP大包下载总时长(s)','HTTP大包下载速率(Mbps)', 'HTTP大包业务低速率话单数(小于2.5Mbps话单数)', 'HTTP大包业务总话单数','HTTP大包业务低速率话单占比(%)', '视频业务下载总业务量(MB)', '视频业务下载总时长(S)','视频业务下载速率(Mbps)','是否质差']]
zc_h.rename(columns={'HTTP大包业务低速率话单数(小于2.5Mbps话单数)':'小于2.5M话单数'},inplace=True)
display(zc_h.sort_values(by=['是否质差','视频业务下载速率(Mbps)'],ascending=False,na_position='first'))

c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2020-06-30 11:00:00 监控小区数35个


,ENB_CI,CELLNAME,质差类型,频段,频点,归属区域,http下载速率(Mbps),视频业务下载速率(Mbps),HTTP大包业务低速率话单占比(%),是否质差
ECI,,,,,,,,,,
63037186,246239_2,洛阳栾川朝阳F-HLH-3,http质差话单占比高,F,38400,县城,3.455,3.948,12.627,1
190909057,745738_129,洛阳孟津汽车检测站F-HLH-2,http质差话单占比高,F,38400,县城,3.801,3.777,11.302,1
26227778,102452_66,洛阳伊川总部大厦F-HLH-3,http质差话单占比高,F,38400,县城,3.631,3.451,13.264,1
70024336,273532_144,洛阳孟津城东市场单路E-HLW,http质差话单占比高,E,39148,县城,2.158,3.025,50.120,1
26318658,102807_66,洛阳西工刘富村南3DMIMOD-HLH-3,http质差话单占比高,D,40936,市区,2.556,2.784,23.044,1
60617730,236788_2,洛阳西工伊众牛肉F-HLH-3,http质差话单占比高,F,38352,市区,2.848,2.743,19.429,1
2891136,11293_128,洛阳偃师诸葛安置小区东北F-HLH-1,http质差话单占比高,F,38400,乡镇,2.543,2.707,22.980,1
234165582,914709_78,洛阳孟津城西FDD1800-HLH-3,http质差话单占比高,FDD1800M,1300,县城,2.467,2.696,18.987,1
63034892,246230_12,洛阳宜阳党校FDD1800-HLH-1,http质差话单占比高,FDD1800M,1300,县城,2.920,2.358,19.012,1


c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,ENB_CI,CELLNAME,质差类型,归属区域_x,开始时间,HTTP大包下载总业务量(MB),HTTP大包下载总时长(s),HTTP大包下载速率(Mbps),小于2.5M话单数,HTTP大包业务总话单数,HTTP大包业务低速率话单占比(%),视频业务下载总业务量(MB),视频业务下载总时长(S),视频业务下载速率(Mbps),是否质差
ECI,,,,,,,,,,,,,,,
62936321,245845_1,洛阳洛宁紫竹河畔F-HLH-2,http质差话单占比高,县城,2020-06-30 11:00:00,105.777,398.554,2.123,9,56,16.071,58.031,55.838,8.314,1
3050122,11914_138,洛阳西工中油一建-HFHH-2,http质差话单占比高,市区,2020-06-30 11:00:00,631.640,943.658,5.355,44,381,11.549,293.391,427.912,5.485,1
70024336,273532_144,洛阳孟津城东市场单路E-HLW,http质差话单占比高,县城,2020-06-30 11:00:00,55.606,179.806,2.474,15,38,39.474,14.208,20.743,5.480,1
69951873,273249_129,洛阳涧西白营南F-HLH-2,http质差话单占比高,市区,2020-06-30 11:00:00,1037.205,2521.196,3.291,100,701,14.265,333.061,632.709,4.211,1
60617015,236785_55,洛阳偃师公路局-HFHH-2,http质差话单占比高,县城,2020-06-30 11:00:00,68.318,144.749,3.776,17,53,32.075,29.735,65.603,3.626,1
63037186,246239_2,洛阳栾川朝阳F-HLH-3,http质差话单占比高,县城,2020-06-30 11:00:00,738.673,1800.418,3.282,77,569,13.533,483.493,1199.582,3.224,1
62981376,246021_0,洛阳西工岳村西F-HLH-1,http质差话单占比高,市区,2020-06-30 11:00:00,1077.849,2673.756,3.225,105,604,17.384,576.995,1503.017,3.071,1
26227778,102452_66,洛阳伊川总部大厦F-HLH-3,http质差话单占比高,县城,2020-06-30 11:00:00,3799.413,10732.922,2.832,471,2219,21.226,1707.788,4575.740,2.986,1
70077325,273739_141,洛阳汝阳液压厂FDD1800-HLH-2,http质差话单占比高,县城,2020-06-30 11:00:00,2863.691,9196.114,2.491,327,1785,18.319,1376.074,3854.098,2.856,1


Wall time: 7.6 s
